In [39]:
#Comparison maps
#FIXME this makes all of them the same statuses??
import os
import glob
import json
import numpy as np
from scipy import stats
import re
import matplotlib.pyplot as plt
import matplotlib.patches as mpatches
import matplotlib.ticker as mticker
from wlt.duts.wafers import WAFER_MAPS
from wlt.wafermap import WaferMap

#init
root_path = os.path.expanduser('~/WLT/WLT_v1.2.1/croc_wlt/data')
path_2 = [entry for entry in os.listdir(root_path) if not entry.startswith('.')]
json_pattern = r'chip_(.*).json'
chip_statuses = {}
chip_col = ''
chip_row = ''
grade = ''

for item in path_2:
    wafer_folder = os.path.join(root_path, item)
    directories = os.listdir(wafer_folder)
    most_recent_subdir = None
    most_recent_time = datetime.min
    for folder in directories:
        if os.path.isdir(os.path.join(wafer_folder, folder)):
            full_path = os.path.join(wafer_folder, folder)
            modified_time = datetime.fromtimestamp(os.path.getmtime(full_path))
            if modified_time > most_recent_time:
                most_recent_time = modified_time
                most_recent_subdir = full_path
                plots_db_path = os.path.join(most_recent_subdir, 'plots', 'db')
                for root, dirs, files in os.walk(plots_db_path):
                    for filename in files:
                        #Find json files
                        if re.search(pattern, filename):
                            filepath = os.path.join(directory, filename)
                            # Open and process JSON file
                            with open(filepath, 'r') as file:
                                data = json.load(file)
                                for k, v in data.items():
                                    if k=="CHIP_COL" and v is not None:
                                        chip_col = v
                                    if k=="CHIP_ROW" and v is not None:
                                        chip_row = v
                                    if k == "GRADE" and v is not None:
                                        grade = v
                                        #changing colors to numbers
                                        if grade == 'red':
                                            grade = 2
                                        elif grade == 'yellow':
                                            grade = 1
                                        elif grade == 'green':
                                            grade = 0
                                        else:
                                            print('unrecognized grade')
                                            grade = -1
                                if chip_col != '' and chip_row !=  '' and grade != '':
                                    chip_label = '(' + str(chip_col) + ',' + str(chip_row) + ')'
                                    print(f'{chip_label}, {grade}, {item}')
                                    if chip_label in chip_statuses:
                                        # Append to the existing list
                                        if isinstance(chip_statuses[chip_label], list):
                                            chip_statuses[chip_label].append(grade)
                                        else:
                                            chip_statuses[chip_label] = [chip_statuses[chip_label], grade]  # Convert to list if it's a string
                                    else:
                                        # Initialize as a list and append the grade
                                        chip_statuses[chip_label] = [grade]

                            
print(chip_statuses)

#next:
#

'''                            


#sort chips into statuses
low = []
medlow = []
medhigh = []
high = []
low_c = []
medlow_c = []
medhigh_c = []
high_c = []
for i in range(len(contactheight)):
    value_height = contactheight[i]
    value_chip = chiplocations[i]
    if value_height < min_height + bin_size:
        low.append(value_chip)
        low_c.append(value_height)
    elif value_height < min_height + 2 * bin_size:
        medlow.append(value_chip)
        medlow_c.append(value_height)
    elif value_height < min_height + 3 * bin_size:
        medhigh.append(value_chip)
        medhigh_c.append(value_height)
    else:
        high.append(value_chip)
        high_c.append(value_height)

#find out which chips are at the median, max, and min height
median = np.median(contactheight)
if median not in contactheight:
    greater_than_median = [num for num in contactheight if num > median]
    next_highest = min(greater_than_median)
'''
'''number of chips at next highest value above median'''
'''
median_chips = []
next_chips = []
max_chips = []
min_chips = []
for i in range(len(contactheight)):
    if contactheight[i] == median:
        median_chips.append(chipids[i])
    if contactheight[i] == next_highest:
        next_chips.append(chipids[i])
    if contactheight[i] == max_height:
        max_chips.append(chipids[i])
    if contactheight[i] == min_height:
        min_chips.append(chipids[i])

#make dictionaries
low_dict = {k:v for k,v in zip(low,low_c)}
medlow_dict = {k:v for k,v in zip(medlow,medlow_c)}
medhigh_dict = {k:v for k,v in zip(medhigh,medhigh_c)}
high_dict = {k:v for k,v in zip(high,high_c)}
'''
#set statuses
chip_statuses = {
		( 1, ''): low_dict,    
		( 2, ''): medlow_dict,                
		( 3, ''): medhigh_dict, 
		( 4, ''): high_dict,              
    }

WaferMap.STATUS_COLORS = {-1: '#aaa', 0: '#06d6a0', 1: '#ffd166', 2: '#ef476f'}
WaferMap.STATUS_NAMES =  {-1: '-',    0: 'g',       1: 'y',       2: 'r'}

#Create map
loop = 0
ch_type ='CROCv2'
wafer_map = WaferMap(chip_type=ch_type, title=f'{ch_type} Contact Height (range: {min_height}um to {max_height}um)')
for (chip_status, chip_value), x in chip_statuses.items():
    for chips,heights in x.items():
        subtext = f"{heights}um"
        wafer_map.set_chip(eval(chips), chip_status, subtext)
        loop = loop + 1
'''
#printing stats
print(f'Range: {min_height}um to {max_height}um ({max_height-min_height}um)')
print(f'Bins:\n - Low (l):          {min_height}-{int(min_height+bin_size)}um\n - Medium low (ml):  {int(min_height+bin_size)}-{int(min_height+bin_size*2)}um\n - Medium high (mh): {int(min_height+bin_size*2)}-{int(min_height+bin_size*3)}um\n - High (h):         {int(min_height+bin_size*3)}-{int(max_height)}um')
print(f'Mean: {round(np.mean(contactheight),3)}um')
print(f'Median: {int(np.median(contactheight))}um')
print(f'Mode: {stats.mode(contactheight)[0]}um')
if median in contactheight:
    print(f'Chips at median height: {", ".join(median_chips)}')
elif median not in contactheight:
    print(f'Calculated median not in data set. Chips at next highest height ({next_highest}um): {next_chips}')
print(f'Chips at maximum height: {", ".join(max_chips)}')
print(f'Chips at minimum height: {", ".join(min_chips)}')
'''

(10,9), 0, wafer_NC0W14-07G5
(7,1), 2, wafer_NC0W14-07G5
(7,3), 0, wafer_NC0W14-07G5
(2,11), 0, wafer_NC0W14-07G5
(10,12), 0, wafer_NC0W14-07G5
(4,5), 0, wafer_NC0W14-07G5
(4,11), 2, wafer_NC0W14-07G5
(11,3), 0, wafer_NC0W14-07G5
(8,10), 0, wafer_NC0W14-07G5
(10,10), 0, wafer_NC0W14-07G5
(6,4), 0, wafer_NC0W14-07G5
(12,9), 0, wafer_NC0W14-07G5
(11,12), 0, wafer_NC0W14-07G5
(9,9), 0, wafer_NC0W14-07G5
(8,13), 2, wafer_NC0W14-07G5
(8,1), 2, wafer_NC0W14-07G5
(12,5), 0, wafer_NC0W14-07G5
(10,5), 0, wafer_NC0W14-07G5
(8,8), 0, wafer_NC0W14-07G5
(10,8), 0, wafer_NC0W14-07G5
(11,7), 0, wafer_NC0W14-07G5
(6,5), 0, wafer_NC0W14-07G5
(5,7), 0, wafer_NC0W14-07G5
(9,3), 0, wafer_NC0W14-07G5
(9,14), 0, wafer_NC0W14-07G5
(11,13), 2, wafer_NC0W14-07G5
(12,7), 0, wafer_NC0W14-07G5
(5,13), 0, wafer_NC0W14-07G5
(4,6), 0, wafer_NC0W14-07G5
(7,4), 2, wafer_NC0W14-07G5
(6,6), 0, wafer_NC0W14-07G5
(11,11), 0, wafer_NC0W14-07G5
(2,6), 0, wafer_NC0W14-07G5
(3,13), 0, wafer_NC0W14-07G5
(3,12), 2, wafer_NC0W14

NameError: name 'low_dict' is not defined

In [32]:
import os
from datetime import datetime

root_path = os.path.expanduser('~/WLT/WLT_v1.2.1/croc_wlt/data')
path_2 = [entry for entry in os.listdir(root_path) if not entry.startswith('.')]
json_pattern = r'chip_(.*).json'
chip_statuses = {}
chip_col = ''
chip_row = ''
grade = ''

for item in path_2:
    wafer_folder = os.path.join(root_path, item)
    directories = os.listdir(wafer_folder)
    most_recent_subdir = None
    most_recent_time = datetime.min
    for folder in directories:
        if os.path.isdir(os.path.join(wafer_folder, folder)):
            full_path = os.path.join(wafer_folder, folder)
            modified_time = datetime.fromtimestamp(os.path.getmtime(full_path))
            if modified_time > most_recent_time:
                most_recent_time = modified_time
                most_recent_subdir = full_path
                plots_db_path = os.path.join(most_recent_subdir, 'plots', 'db')
                for root, dirs, files in os.walk(plots_db_path):
                    for filename in files:
                        #Find json files
                        if re.search(pattern, filename):
                            filepath = os.path.join(directory, filename)
                            # Open and process JSON file
                            with open(filepath, 'r') as file:
                                data = json.load(file)
                                for k, v in data.items():
                                    if k=="CHIP_COL" and v is not None:
                                        chip_col = v
                                    if k=="CHIP_ROW" and v is not None:
                                        chip_row = v
                                    if k == "GRADE" and v is not None:
                                        grade = v
                                        #could change colors to numbers here?
                                if chip_col != '' and chip_row !=  '' and grade != '':
                                    chip_label = str(chip_col) + ',' + str(chip_row)
                                    if chip_label in chip_statuses:
                                        # Append to the existing list
                                        if isinstance(chip_statuses[chip_label], list):
                                            chip_statuses[chip_label].append(grade)
                                        else:
                                            chip_statuses[chip_label] = [chip_statuses[chip_label], grade]  # Convert to list if it's a string
                                    else:
                                        # Initialize as a list and append the grade
                                        chip_statuses[chip_label] = [grade]

                            
print(chip_statuses)
    
    
    
    
    
'''  
#initialize
root_dir = os.path.expanduser('~/WLT/WLT_v1.2.1/croc_wlt/data')
most_recent_dir = None
most_recent_time = datetime.min


#find most recent subdirectory
most_recent_subdir = None
most_recent_time = datetime.min
if most_recent_dir:
    for folder in os.listdir(most_recent_dir):
        if os.path.isdir(os.path.join(most_recent_dir, folder)):
            full_path = os.path.join(most_recent_dir, folder)
            modified_time = datetime.fromtimestamp(os.path.getmtime(full_path))
            if modified_time > most_recent_time:
                most_recent_time = modified_time
                most_recent_subdir = full_path
                print('most recent: ' + str(most_recent_subdir))
                plots_db_path = os.path.join(most_recent_subdir, 'plots', 'db')
                print(plots_db_path)
                for root, dirs, files in os.walk(plots_db_path):
                    for file in files:
                        file_path = os.path.join(root, file)
                        # Process each file here
                        print(f'Processing file: {file_path}')

#find most recent log file
most_recent_file = None
most_recent_time = None

# Iterate through all files in the directory
for filename in os.listdir(most_recent_subdir):
    if filename.endswith(".log"):  # Check if file ends with .log
        file_path = os.path.join(most_recent_subdir, filename)
        modification_time = os.path.getmtime(file_path)

        # Update most_recent_file and most_recent_time if current file is more recent
        if most_recent_time is None or modification_time > most_recent_time:
            most_recent_time = modification_time
            most_recent_file = file_path

# Check if a most recent log file was found
if most_recent_file:
    #print(f"The most recent log file is: {most_recent_file}")
    #print(f"Last modified time: {datetime.fromtimestamp(most_recent_time)}")
    pass
else:
    print("No log files found in the directory.")
with open(most_recent_file, 'r') as file:
    log_content = file.readlines()
return log_content
'''

{'10,9': ['green', 'green', 'green', 'green', 'green'], '7,1': ['red', 'red', 'red', 'red', 'red'], '7,3': ['green', 'green', 'green', 'green', 'green'], '2,11': ['green', 'green', 'green', 'green', 'green'], '10,12': ['green', 'green', 'green', 'green', 'green'], '4,5': ['green', 'green', 'green', 'green', 'green'], '4,11': ['red', 'red', 'red', 'red', 'red'], '11,3': ['green', 'green', 'green', 'green', 'green'], '8,10': ['green', 'green', 'green', 'green', 'green'], '10,10': ['green', 'green', 'green', 'green', 'green'], '6,4': ['green', 'green', 'green', 'green', 'green'], '12,9': ['green', 'green', 'green', 'green', 'green'], '11,12': ['green', 'green', 'green', 'green', 'green'], '9,9': ['green', 'green', 'green', 'green', 'green'], '8,13': ['red', 'red', 'red', 'red', 'red'], '8,1': ['red', 'red', 'red', 'red', 'red'], '12,5': ['green', 'green', 'green', 'green', 'green'], '10,5': ['green', 'green', 'green', 'green', 'green'], '8,8': ['green', 'green', 'green', 'green', 'green']

'  \n#initialize\nroot_dir = os.path.expanduser(\'~/WLT/WLT_v1.2.1/croc_wlt/data\')\nmost_recent_dir = None\nmost_recent_time = datetime.min\n\n\n#find most recent subdirectory\nmost_recent_subdir = None\nmost_recent_time = datetime.min\nif most_recent_dir:\n    for folder in os.listdir(most_recent_dir):\n        if os.path.isdir(os.path.join(most_recent_dir, folder)):\n            full_path = os.path.join(most_recent_dir, folder)\n            modified_time = datetime.fromtimestamp(os.path.getmtime(full_path))\n            if modified_time > most_recent_time:\n                most_recent_time = modified_time\n                most_recent_subdir = full_path\n                print(\'most recent: \' + str(most_recent_subdir))\n                plots_db_path = os.path.join(most_recent_subdir, \'plots\', \'db\')\n                print(plots_db_path)\n                for root, dirs, files in os.walk(plots_db_path):\n                    for file in files:\n                        file_path = os